In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import os
import re
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import shutil
import random
from tqdm import tqdm

2025-10-12 20:19:38.924484: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-12 20:19:38.954395: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-12 20:19:39.677046: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Reduz logs do TensorFlow
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/lib/cuda'  # Configuração XLA

import tensorflow as tf

tf.config.optimizer.set_jit(False)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU disponível: {len(gpus)} GPU(s) encontrada(s)")
        print(f"   JIT compilation: Desabilitado (para evitar erros)")
    except RuntimeError as e:
        print(f"⚠️ Erro ao configurar GPU: {e}")
else:
    print("⚠️ Nenhuma GPU encontrada. Usando CPU.")

print(f"TensorFlow version: {tf.__version__}")

✅ GPU disponível: 1 GPU(s) encontrada(s)
   JIT compilation: Desabilitado (para evitar erros)
TensorFlow version: 2.20.0


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [4]:
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
path2 = ROOT_DIR + '/data/muhammad0subhan/fruit-and-vegetable-disease-healthy-vs-rotten/versions/1/Fruit And Vegetable Diseases Dataset'

In [5]:
full = os.listdir(path2)

In [13]:
full

['Carrot__Healthy',
 'Potato__Healthy',
 'Tomato__Healthy',
 'Carrot__Rotten',
 'Tomato__Rotten',
 'Potato__Rotten']

In [6]:
def extract_fruit_name(name):
    name = name.lower()
    name = re.sub(r'^(fresh|rotten)', '', name)
    name = re.sub(r'(s)$','',name)
    name = re.sub(r'(__healthy|__rotten)$', '', name)
    return name.strip()
cleaned_fruits = set([extract_fruit_name(fruit) for fruit in full])
print("Unique fruits found:", len(cleaned_fruits))
print(sorted(cleaned_fruits))

Unique fruits found: 3
['carrot', 'potato', 'tomato']


In [7]:
freshfruit_dict = {fruit: 0 for fruit in cleaned_fruits}
rottenfruit_dict = {fruit: 0 for fruit in cleaned_fruits}
print(freshfruit_dict)

{'carrot': 0, 'potato': 0, 'tomato': 0}


In [14]:
for i in full :
    if re.sub(r'(__Rotten)$','',i).lower() in cleaned_fruits:
        rottenfruit_dict[re.sub(r'(__Rotten)$','',i).lower()] += len(os.listdir(os.path.join(path2,i)))
        print(i,' : ',len(os.listdir(os.path.join(path2,i))))

Carrot__Rotten  :  580
Tomato__Rotten  :  596
Potato__Rotten  :  585


In [15]:
for i in full :
    if re.sub(r'(__Healthy)$','',i).lower() in cleaned_fruits:
        freshfruit_dict[re.sub(r'(__Healthy)$','',i).lower()] += len(os.listdir(os.path.join(path2,i)))
        print(i,' : ',len(os.listdir(os.path.join(path2,i))))

Carrot__Healthy  :  620
Potato__Healthy  :  615
Tomato__Healthy  :  604


In [16]:
freshfruit_dict

{'carrot': 620, 'potato': 615, 'tomato': 604}

In [17]:
rottenfruit_dict

{'carrot': 580, 'potato': 585, 'tomato': 596}

In [ ]:
def generate_augmented_images_limited_per_image(src_folder, dst_folder, num_images, image_size=(224, 224), per_image_limit=20):
    """
    Generate augmented images, limiting to `per_image_limit` per source image.

    Args:
        src_folder (str): Path to source folder with original images.
        dst_folder (str): Destination folder for augmented images.
        num_images (int): Total augmented images to generate.
        image_size (tuple): Resize all images to this size. Default (224, 224).
        per_image_limit (int): Max images to generate per source image. Default 20.
    """
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)

    datagen = ImageDataGenerator(
        rotation_range=25,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    image_files = [f for f in os.listdir(
        src_folder) if f.lower().endswith(('jpg', 'jpeg', 'png'))]
    if not image_files:
        print("❌ No valid images found in source folder.")
        return

    total_generated = 0

    for image_file in image_files:
        if total_generated >= num_images:
            break

        img_path = os.path.join(src_folder, image_file)
        img = load_img(img_path, target_size=image_size)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        images_to_generate = min(per_image_limit, num_images - total_generated)
        count = 0

        for batch in datagen.flow(x, batch_size=1, save_to_dir=dst_folder,
                                  save_prefix='aug', save_format='jpg'):
            count += 1
            total_generated += 1
            if count >= images_to_generate or total_generated >= num_images:
                break

    print(f"✅ Generated {total_generated} augmented images in '{dst_folder}'")

In [23]:
def copy_dataset(src_dir, dst_dir):
    """
    Copy the entire folder structure and files from `src_dir` to `dst_dir`.

    Args:
        src_dir (str): Path to the original dataset.
        dst_dir (str): Path to the new directory where the dataset will be copied.
    """
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    for folder_name in os.listdir(src_dir):
        src_folder_path = os.path.join(src_dir, folder_name)
        dst_folder_path = os.path.join(dst_dir, folder_name)

        if os.path.isdir(src_folder_path):
            shutil.copytree(src_folder_path, dst_folder_path,
                            dirs_exist_ok=True)
            print(f"✅ Copied: {folder_name}")
        else:
            print(f"❌ Skipped (not a folder): {folder_name}")

In [ ]:
dst_dir = ROOT_DIR + '/data/working'
copy_dataset(path2, dst_dir)

In [29]:
for i in full:
    if len(os.listdir(os.path.join(dst_dir, i))) < 1000:
        print(i, ' : ', len(os.listdir(os.path.join(path2, i))))
        generate_augmented_images_limited_per_image(os.path.join(dst_dir, i), os.path.join(
            dst_dir, i), 1000 - len(os.listdir(os.path.join(path2, i))), image_size=(224, 224), per_image_limit=20)

Carrot__Healthy  :  620
✅ Generated 380 augmented images in '/home/dev/repos/diogo/visao_computacional/data/working/Carrot__Healthy'
Potato__Healthy  :  615
✅ Generated 385 augmented images in '/home/dev/repos/diogo/visao_computacional/data/working/Potato__Healthy'
Tomato__Healthy  :  604
✅ Generated 396 augmented images in '/home/dev/repos/diogo/visao_computacional/data/working/Tomato__Healthy'
Carrot__Rotten  :  580
✅ Generated 420 augmented images in '/home/dev/repos/diogo/visao_computacional/data/working/Carrot__Rotten'
Tomato__Rotten  :  596
✅ Generated 404 augmented images in '/home/dev/repos/diogo/visao_computacional/data/working/Tomato__Rotten'
Potato__Rotten  :  585
✅ Generated 415 augmented images in '/home/dev/repos/diogo/visao_computacional/data/working/Potato__Rotten'


In [30]:
def ensure_image_size(img_path, target_size=(224, 224)):
    try:
        with Image.open(img_path) as img:
            img = img.convert('RGB')  # Convert from RGBA to RGB
            if img.size != target_size:
                img = img.resize(target_size)
            img.save(img_path)  # Overwrite
    except Exception as e:
        print(f"Error resizing {img_path}: {e}")


def split_and_resize_dataset(src_dir, dst_dir, target_size=(224, 224), train_count=850, val_count=100):
    os.makedirs(dst_dir, exist_ok=True)

    for class_name in tqdm(os.listdir(src_dir)):
        class_path = os.path.join(src_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        images = [f for f in os.listdir(
            class_path) if f.lower().endswith(('jpg', 'jpeg', 'png'))]
        random.shuffle(images)

        train_images = images[:train_count]
        val_images = images[train_count:train_count+val_count]

        for phase, image_set in zip(['train', 'val'], [train_images, val_images]):
            phase_class_path = os.path.join(dst_dir, phase, class_name)
            os.makedirs(phase_class_path, exist_ok=True)

            for img_name in image_set:
                src_img_path = os.path.join(class_path, img_name)
                dst_img_path = os.path.join(phase_class_path, img_name)

                ensure_image_size(src_img_path, target_size)
                shutil.copy(src_img_path, dst_img_path)


# Example usage
split_and_resize_dataset(
    src_dir=dst_dir,
    dst_dir=dst_dir,
    target_size=(224, 224)
)

 17%|█▋        | 1/6 [00:02<00:11,  2.33s/it]/home/dev/repos/diogo/visao_computacional/.venv/lib/python3.11/site-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 6/6 [00:12<00:00,  2.15s/it]


In [31]:
os.listdir(dst_dir + '/train')

['Carrot__Healthy',
 'Potato__Healthy',
 'Tomato__Healthy',
 'Carrot__Rotten',
 'Tomato__Rotten',
 'Potato__Rotten']

In [7]:
train_dir = dst_dir + "/train"
val_dir = dst_dir + "/val"

img_size = 224
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 5100 images belonging to 6 classes.


Found 600 images belonging to 6 classes.


In [8]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = True # Freeze base layers

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    run_eagerly=True  # This can help with some execution issues
)

I0000 00:00:1760311214.152707  166123 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4028 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
2025-10-12 20:20:14.286146: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-12 20:20:14.289707: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-12 20:20:14.293008: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-12 20:20:14.296920: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. F

In [9]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=6  # increase if needed
)

Epoch 1/6


/home/dev/repos/diogo/visao_computacional/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2025-10-12 20:20:18.419080: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
2025-10-12 20:20:18.637600: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-12 20:20:18.639078: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-12 20:20:18.640390: W tensorflow/compiler/mlir/tools/kernel_gen/trans

  1/319 ━━━━━━━━━━━━━━━━━━━━ 13:47 3s/step - accuracy: 0.2500 - loss: 1.9092

2025-10-12 20:20:20.811920: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-12 20:20:20.814891: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-12 20:20:20.817972: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


319/319 ━━━━━━━━━━━━━━━━━━━━ 150s 463ms/step - accuracy: 0.8992 - loss: 0.3033 - val_accuracy: 0.8067 - val_loss: 0.6195
Epoch 2/6
319/319 ━━━━━━━━━━━━━━━━━━━━ 147s 461ms/step - accuracy: 0.9716 - loss: 0.0881 - val_accuracy: 0.9150 - val_loss: 0.2548
Epoch 3/6
319/319 ━━━━━━━━━━━━━━━━━━━━ 147s 461ms/step - accuracy: 0.9794 - loss: 0.0646 - val_accuracy: 0.9433 - val_loss: 0.1974
Epoch 4/6
319/319 ━━━━━━━━━━━━━━━━━━━━ 147s 462ms/step - accuracy: 0.9859 - loss: 0.0416 - val_accuracy: 0.9583 - val_loss: 0.1493
Epoch 5/6
319/319 ━━━━━━━━━━━━━━━━━━━━ 148s 464ms/step - accuracy: 0.9867 - loss: 0.0389 - val_accuracy: 0.9767 - val_loss: 0.0945
Epoch 6/6
319/319 ━━━━━━━━━━━━━━━━━━━━ 147s 462ms/step - accuracy: 0.9920 - loss: 0.0249 - val_accuracy: 0.9617 - val_loss: 0.1396
